In [1]:
import liana as li
import scanpy as sc
from anndata import AnnData
import pandas as pd
from liana.mt import rank_aggregate
from liana.method import singlecellsignalr, connectome, cellphonedb, natmi, logfc, cellchat, geometric_mean
from pycirclize import Circos

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [2]:
# loading original anndata
pvat_8weeks_hf_female = sc.read_h5ad('../../exported_data/pvat_8weeks_hf_female.h5ad')
pvat_8weeks_hf_female

AnnData object with n_obs × n_vars = 6314 × 20743
    obs: 'sample_id', 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'pct_counts_in_top_20_genes', 'total_counts_mt', 'log1p_total_counts_mt', 'pct_counts_mt', 'total_counts_ribo', 'log1p_total_counts_ribo', 'pct_counts_ribo', 'total_counts_hb', 'log1p_total_counts_hb', 'pct_counts_hb', 'outlier', 'mt_outlier', 'n_genes', 'scDblFinder_score', 'scDblFinder_class', 'Sample', 'tissue', 'diet', 'time', 'sex', 'batch', 'sample_type', 'rat_id', 'weight_3W', 'weight_10W', 'weight_17W', 'weight_24W', 'MAP_7W', 'systolic_bp_7W', 'MAP_9W', 'systolic_bp_9W', 'MAP_16W', 'systolic_bp_16W', 'MAP_24W', 'systolic_bp_24W', 'doublet', 'celltype', 'celltype_broad'
    var: 'gene_ids', 'mt', 'ribo', 'hb', 'n_cells_by_counts', 'mean_counts', 'log1p_mean_counts', 'pct_dropout_by_counts', 'total_counts', 'log1p_total_counts', 'n_cells'
    uns: 'cellchat_interaction_table_count', 'cellchat_res', 'cellchat_res_filtered', 

In [3]:
pvat_8weeks_hf_female.obs['celltype_broad'].unique().tolist()

['Adipocytes',
 'Fibroblasts',
 'Endothelial_Cells',
 'Immune_Cells',
 'Pericytes',
 'Mesothelial_Cells',
 'SMCs',
 'Neuronal_Cells']

In [4]:
# List available methods in lliana framework
li.mt.show_methods()

,Method Name,Magnitude Score,Specificity Score,Reference
0,CellPhoneDB,lr_means,cellphone_pvals,"Efremova, M., Vento-Tormo, M., Teichmann, S.A...."
0,Connectome,expr_prod,scaled_weight,"Raredon, M.S.B., Yang, J., Garritano, J., Wang..."
0,log2FC,None,lr_logfc,"Dimitrov, D., Türei, D., Garrido-Rodriguez, M...."
0,NATMI,expr_prod,spec_weight,"Hou, R., Denisenko, E., Ong, H.T., Ramilowski,..."
0,SingleCellSignalR,lrscore,None,"Cabello-Aguilar, S., Alame, M., Kon-Sun-Tack, ..."
0,CellChat,lr_probs,cellchat_pvals,"Jin, S., Guerrero-Juarez, C.F., Zhang, L., Cha..."
0,Rank_Aggregate,magnitude_rank,specificity_rank,"Dimitrov, D., Türei, D., Garrido-Rodriguez, M...."
0,Geometric Mean,lr_gmeans,gmean_pvals,CellPhoneDBv2's permutation approach applied t...


In [5]:
pvat_8weeks_hf_female.raw = pvat_8weeks_hf_female
pvat_8weeks_hf_female.raw.X

<6314x20743 sparse matrix of type '<class 'numpy.float64'>'
	with 11552260 stored elements in Compressed Sparse Row format>

In [6]:
# Run cellchat analysis on PVAT data
cellchat(pvat_8weeks_hf_female, groupby='celltype_broad', expr_prop=0.1, resource_name='mouseconsensus', verbose=True, key_added='cellchat_res')

Using `.raw`!
/Users/pandavis/miniconda3/lib/python3.11/site-packages/anndata/_core/anndata.py:522: FutureWarning: The dtype argument is deprecated and will be removed in late 2024.
1598 features of mat are empty, they will be removed.
Make sure that normalized counts are passed!
/Users/pandavis/miniconda3/lib/python3.11/site-packages/liana/method/_pipe_utils/_pre.py:148: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
['Cyp2c6_v1-ps2'] contain `_`. Consider replacing those!
Using resource `mouseconsensus`.
0.21 of entities in the resource are missing from the data.


Generating ligand-receptor stats for 6314 samples and 19145 features


100%|████████████████████████████████████████████████████████████████████████████████████| 1000/1000 [01:38<00:00, 10.12it/s]


In [7]:
ligand_receptor_pvat = pvat_8weeks_hf_female.uns['cellchat_res']
ligand_receptor_pvat.head()

,ligand,ligand_complex,ligand_props,ligand_trimean,mat_max,receptor,receptor_complex,receptor_props,receptor_trimean,source,target,lr_probs,cellchat_pvals
92,Vegfa,Vegfa,0.977831,0.026204,353.0,Nrp1,Nrp1,0.923990,0.024079,Adipocytes,Adipocytes,0.001260,0.000
759,C4a,C4a,0.887850,0.009207,353.0,Nrp1,Nrp1,0.923990,0.024079,Mesothelial_Cells,Adipocytes,0.000443,0.000
1470,Vegfa,Vegfa,0.977831,0.026204,353.0,Nrp1,Nrp1,0.739437,0.007790,Adipocytes,Endothelial_Cells,0.000408,0.089
3136,Vegfa,Vegfa,0.977831,0.026204,353.0,Egfr,Egfr,0.775689,0.007790,Adipocytes,Fibroblasts,0.000408,0.000
11726,Ntf3,Ntf3,0.636364,0.004603,353.0,Ntrk3,Ntrk3,1.000000,0.042847,SMCs,SMCs,0.000394,0.000


In [8]:
ligand_receptor_string = (ligand_receptor_pvat.loc[:,'ligand_complex'].str.contains('Col')) & (ligand_receptor_pvat.loc[:,'receptor_complex'].str.contains('Itg'))
cell_source_string = ligand_receptor_pvat.loc[:,'source'] == 'Fibroblasts'
cell_target_string = ligand_receptor_pvat.loc[:,'target'] == 'Adipocytes'
cell_pvals = ligand_receptor_pvat.loc[:,'cellchat_pvals'] <= 0.05

ligand_receptor_pvat[ligand_receptor_string & cell_source_string & cell_target_string & cell_pvals]

,ligand,ligand_complex,ligand_props,ligand_trimean,mat_max,receptor,receptor_complex,receptor_props,receptor_trimean,source,target,lr_probs,cellchat_pvals
319,Col4a1,Col4a1,0.778195,0.014164,353.0,Itgb1,Itga1_Itgb1,0.477963,0.000708,Fibroblasts,Adipocytes,0.000020,0.0
325,Col4a2,Col4a2,0.753133,0.013456,353.0,Itgb1,Itga1_Itgb1,0.477963,0.000708,Fibroblasts,Adipocytes,0.000019,0.0
347,Col5a3,Col5a3,0.820802,0.012571,353.0,Itgb1,Itga1_Itgb1,0.477963,0.000708,Fibroblasts,Adipocytes,0.000018,0.0
353,Col3a1,Col3a1,0.878446,0.009915,353.0,Itgb1,Itga1_Itgb1,0.477963,0.000708,Fibroblasts,Adipocytes,0.000014,0.0
331,Col6a3,Col6a3,0.766917,0.007790,353.0,Itgb1,Itga1_Itgb1,0.477963,0.000708,Fibroblasts,Adipocytes,0.000011,0.0
329,Col6a2,Col6a2,0.770677,0.006374,353.0,Itgb1,Itga1_Itgb1,0.477963,0.000708,Fibroblasts,Adipocytes,0.000009,0.0
349,Col5a2,Col5a2,0.733083,0.005666,353.0,Itgb1,Itga1_Itgb1,0.477963,0.000708,Fibroblasts,Adipocytes,0.000008,0.0
343,Col15a1,Col15a1,0.627820,0.004249,353.0,Itgb1,Itga1_Itgb1,0.477963,0.000708,Fibroblasts,Adipocytes,0.000006,0.0
313,Col6a6,Col6a6,0.562657,0.003541,353.0,Itgb1,Itga1_Itgb1,0.477963,0.000708,Fibroblasts,Adipocytes,0.000005,0.0
327,Col6a1,Col6a1,0.684211,0.003541,353.0,Itgb1,Itga1_Itgb1,0.477963,0.000708,Fibroblasts,Adipocytes,0.000005,0.0


In [9]:
ligand_receptor_pvat.to_csv("pvat_8weeks_hf_female_cellchat.csv")
pvat_8weeks_hf_female.write("../../exported_data/pvat_8weeks_hf_female.h5ad")